# PGFinder Interactive Notebook

This notebook is a basic user interface to allow researchers less familiar with the command line to run PGFinder. Some compromises have been made to make it run as a Jupyter notebook on the free myBinder platform.

To use the code to analyse your data you must work from top to bottom on this notebook, following instructions as you go. The first step is to execute all of the "cells" in the notebook so they are ready for your input. To do this...

**Click *Kernel*>*Restart & Run All* on the menu, above.**

If you have any problems or suggestions, or would like to contribute a mass library, please raise an [issue here](https://github.com/Mesnage-Org/pgfinder/issues).

In [ ]:
import base64
import codecs
import io
import os
import uuid
import ipywidgets as widgets
from ipywidgets import HTML
from IPython.display import display
from ipysheet import from_dataframe
import pandas as pd

from pgfinder import matching, pgio, validation

# Get list of modifications
allowed_mods = validation.allowed_modifications()

# Get built in mass lists
mass_lists_path = './data/masses'
mass_lists = os.listdir(mass_lists_path)
mass_lists.append('Upload Custom')

# Main analysis function
def analysis(b):
    # Upload deconvoluted file
    uploaded_df = pgio.ms_upload_reader(data_uploader.value)
    
    # Load mass list
    if rb_masses.value == 'Upload Custom':
        theo_masses = pgio.theo_masses_upload_reader(mass_uploader.value)
    else:
        csv_filepath = os.path.join(mass_lists_path, rb_masses.value)
        theo_masses = pgio.theo_masses_reader(csv_filepath)

    # Load ppm value
    user_ppm = ppm_tol.value

    # Load time delta value
    user_time_delta = time_delta.value

    # Make sure mod list is a list
    mod_list = list(selector_mods.value)
    
    results = matching.data_analysis(uploaded_df, theo_masses, user_time_delta, mod_list, user_ppm)
    
    # Make the download button
    results_csv_str = results.to_csv()
    filename = pgio.default_filename()
    results_csv_str = pgio.dataframe_to_csv_metadata(output_dataframe=results)
    b64 = base64.b64encode(results_csv_str.encode())
    payload = b64.decode()

    html_buttons = '''<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="{filename}" href="data:text/csv;base64,{payload}" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
    </a>
    </body>
    </html>
    '''

    html_button = html_buttons.format(payload=payload,filename=filename)
    display(HTML(html_button))
    
# Define widgets

# Layout for a bigger button
big_button = widgets.Layout(width='auto')

# Style for wider description
wide_style = {'description_width': 'initial'}

# Deconvoluted Data file upload
data_uploader = widgets.FileUpload(
    accept = '.txt,.ftrs', 
    description = 'Upload Deconvoluted Data',
    multiple = False,
    layout = big_button
) 

# Modifcation selector
selector_mods = widgets.SelectMultiple(
    options = allowed_mods,
    description = 'Modification',
    disabled = False
)

# Mass library selector
rb_masses = widgets.RadioButtons(
    options = mass_lists,
    description = 'Mass List',
    disabled = False
)

# Mass library file upload
mass_uploader = widgets.FileUpload(
    accept = '.csv', 
    description = 'Upload Mass Library',
    multiple = False,
    layout = big_button
)

# Set PPM tolerance
ppm_tol = widgets.BoundedFloatText(
    value = 10.0,
    min = 1,
    max = 100,
    step = 0.1,
    description = 'Set ppm tolerance',
    disabled = False,
    style = wide_style
    )

# Set time delta for in source clean up
time_delta = widgets.BoundedFloatText(
    value = 0.5,
    min = 0,
    max = 100,
    step = 0.01,
    description = 'Set time delta value',
    disabled = False,
    style = wide_style
)

# Analysis button
button = widgets.Button(description="Run Analysis")
button.on_click(analysis)

## Step 1: Upload Deconvoluted Data
Click *Upload* to upload a `.txt` file output by MaxQuant ([example file](https://github.com/Mesnage-Org/pgfinder/raw/master/data/maxquant_test_data.txt)), or an `.ftrs` file.

In [ ]:
display(data_uploader)

## Step 2: Select Modifications
Select modifications (Hold down control / command and click to select mulitple items.)

In [ ]:
display(selector_mods) 

## Step 3: Select or Upload Mass Library

### Select

In [ ]:
display(rb_masses)

### (Optional) Upload Custom Mass Library
[Example mass library file.](https://raw.githubusercontent.com/Mesnage-Org/pgfinder/master/data/masses/e_coli_monomer_masses.csv)

In [ ]:
display(mass_uploader)

## Step 4: Set PPM tolerance

In [ ]:
display(ppm_tol)

## Step 5: Set time window for in-source decay and salt adduct clean up

In [ ]:
display(time_delta)

## Step 6: Run Analysis
Click run analysis.
After the analysis is complete, a download button will appear.

In [ ]:
display(button)